# Radon-Cumulative Distribution Transform Nearest Subspace (RCDT-NS) Classifier

This tutorial will demonstrate how to use the RCDT-NS classifier in the *PyTransKit* package.

## Class:: INV_ENC
**Functions**:

1. Constructor function:
        rcdt_ns_obj = INV_ENC(num_classes, thetas, rm_edge)
        
        Inputs:
        ----------------
        num_classes : integer value
            totale number of classes in the dataset.
        thetas : 1d array
            angles in degrees for taking radon projections. Example: thetas=numpy.linspace(0,180,45)
        rm_edge : boolean 
            IF TRUE the first and last points of RCDTs will be removed.
            
        Outputs:
        ----------------
        rcdt_ns_obj : class object
            Instance of the class INV_ENC.
            
2. Fit function:
        rcdt_ns_obj.fit(Xtrain, Ytrain, no_deform_model)
        
        Inputs:
        ----------------
        Xtrain : 3d array, shape (n_samples, n_rows, n_columns)
            Image data for training.
        Ytrain : 1d array, shape (n_samples,)
            Labels of the training images.
        no_deform_model : boolean
            IF TRUE, no deformation model will be added
  
3. Predict function:
        preds = rcdt_ns_obj.predict(Xtest, use_gpu)
        
        Inputs:
        ----------------
        Xtest : 3d array, shape (n_samples, n_rows, n_columns)
            Image data for testing.
        use_gpu : boolean 
            IF TRUE, use gpu for calculations.
            
        Outputs:
        ----------------
        preds : 1d array, shape (n_samples,)
           Predicted labels for test samples.
    

## Example
The following example will demonstrate how to:
* create and initialize an instance of the class INV_ENC
* train the model with training images
* apply the model to predict calss labels of the test images
In this example we have used MNIST dataset stored in the *data* folder

### Import some python libraries

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys
sys.path.append('../')
from pytranskit.classification.utils import *

use_gpu = False

### Import RCDT-NS class from *PyTransKit* package

In [2]:
from pytranskit.classification.inv_enc import INV_ENC

### Load dataset
For loading data we have used *load_data* function from the *pytranskit/classifier/utils.py* script. It takes name and directory of the dataset, and total number of classes as input. Returns both train and test images in two separate 3d arrays of shape (n_samples, n_rows, n_columns), and corresponding class labels. User can use there own implementation to load data, just need to make sure that the output arrays are consistent.

In [3]:
datadir = './data'
dataset = 'MNIST'
num_classes = 10          # total number of classes in the dataset
(x_train, y_train), (x_test, y_test) = load_data(dataset, num_classes, datadir)  # load_data function from utils.py

loading data from mat files
x_train.shape (60000, 28, 28) x_test.shape (10000, 28, 28)
saved to ./data/MNIST/dataset.hdf5


In this example we have used 512 randomly chosen samples per class to train the model. We have used another function *take_train_samples* function from *utils.py* script for this. User can use their own script.

In [4]:
n_samples_perclass = 512  # total number of training samples per class used in this example
x_train_sub, y_train_sub = take_train_samples(x_train, y_train, n_samples_perclass, 
                                              num_classes, repeat=0) # function from utils.py

### Create an instance of INV_ENC class

In [5]:
theta = np.linspace(0, 176, 45)    # choose the angles in degrees that will be used to calculate Radon projections
rcdt_ns_obj = INV_ENC(num_classes, theta, rm_edge=True)

### Training phase
This function takes the train samples and labels as input, and stores the basis vectors for corresponding classes in a *private* variable. This variable will be used in the *predict* function in the test phase

In [6]:
rcdt_ns_obj.fit(x_train_sub, y_train_sub)


Calculating RCDTs for training images ...
Generating basis vectors for each class ...


### Testing phase
*predict* function takes the train samples as input and returns the predicted class labels

In [7]:
preds = rcdt_ns_obj.predict(x_test, use_gpu)


Calculating RCDTs for testing images ...
Finding nearest subspace for each test sample ...


In [8]:
print('\nTest accuracy: {}%'.format(100*accuracy_score(y_test, preds)))


Test accuracy: 95.41%
